In [1]:
# Animal Game (Jogo do Bicho)

# The Animal Game is a popular lottery-like game in Brazil, played since the 1940s,
# despite being ilegal for most of its existence, up until this day.

# Every lottery draw contains five numbers with four digits each, from '0000' to '9999' and
# players can make different kinds of bets. 

# The most common bet is to pick one out of 25 possible animals. Each animal corresponds to
# a group of four numbers.

# The first animal is the Ostrich (Avestruz), and its numbers are: [01, 02, 03, 04], the next
# is the Eagle (Águia) [05, 06, 07, 08], and so forth, up until the Cow (Vaca) [97, 98, 99, 00].

# A player who bets on a single animal wins if any of the animals four numbers appear in the
# last two digits of any of the five lottery draws. There are many other kinds of bets.

# For instance, the same game described above can be played in 'dry' or 'head' mode, where the
# player only wins if the chosen animal appears in the first out of the five draws. This is five
# times less likely so the payout of this game mode is usually five times greater than the latter.

# Players can also bet on specific numbers instead groups of numbers (animals). The bets are made
# on the last two, three or four digits of the draws. If either of these bets are made in 'dry'
# mode only the first draw of the lot is considered and the prize is five times greater.

# Finally it is also possible to bet on two or three animals, or even 2 double digit numbers.


# What is this thing anyway?

# - A bad excuse to use metaclasses. 
# - A mini framework for easily creating different types of games and running simulations of the Animal Game.
# - A random act of data exploration.

In [2]:
ANIMALS = ('Avestruz', 'Águia', 'Burro', 'Borboleta', 'Cachorro', 'Cabra', 'Carneiro', 'Camelo', 'Cobra', 'Coelho', 'Cavalo', 'Elefante', 'Galo', 'Gato', 'Jacaré', 'Leão', 'Macaco', 'Porco', 'Pavão', 'Peru', 'Touro', 'Tigre', 'Urso', 'Veado', 'Vaca',)

In [3]:
def animal_id_from_digits(digits: str) -> str:
    i = (int(digits[-2:])-1)//4
    if i < 0:
        i = 24
    return str(i).zfill(2)

In [4]:
# The Round class is used to compare a bet choice and a draw.

# The animal flag is used to indicate that the choices represent
# the index of an animal and not a two digit number.

In [5]:
from collections import Counter

class Round:

    def __init__(self, choices: list, draw: list, animal=False):
        self.choices = choices
        if animal:
            self.draw = [animal_id_from_digits(d) for d in draw]
        else:
            self.draw = [d[-len(choices[0]):] for d in draw]

    def check_present(self) -> bool:
        prev_count = sum(Counter(self.draw).values())
        after_removal = sum( (Counter(self.draw) - Counter(self.choices)).values() )
        return prev_count - after_removal == len(self.choices)
    
    def check_ordered(self) -> bool:
        length = len(self.choices)
        try:
            first_idx = self.draw.index(self.choices[0])
        except ValueError:
            return False
        return self.choices == self.draw[first_idx:first_idx+length]
    
    def check_exact(self) -> bool:
        matches = [self.choices[i] == self.draw[i] for i in range(len(self.choices))]
        return all(matches)

In [6]:
# GameMeta is a metaclass.

# It probably did not have to be a metaclass, but it is.
# I blame David Beazley.

# The metaclass will inject into the class the calc, check_win and
# random methods. It will also create one instance of the class
# and keep it in a dictionary.

# The idea behind this madness is to permit the creation of different
# kinds of games by simply declaring a class that contains class variables
# that describe how the game should work.

# The metaclass also provides convinience methods to list all game type instances
# or to grab one by the name of the class.

In [7]:
import numpy as np

class GameMeta(type):

    __type_instances = {}

    def __new__(cls, name, bases, clsdict):

        # Create the class:
        clsobj = super().__new__(cls, name, bases, clsdict)
        
        # Set default class vars if they are not present:
        if not hasattr(clsobj, 'prize'):
            setattr(clsobj, 'prize', 2)
        if not hasattr(clsobj, 'match_index'):
            setattr(clsobj, 'match_index', False)
        if not hasattr(clsobj, 'match_order'):
            setattr(clsobj, 'match_order', False)
        if not hasattr(clsobj, 'n_choices'):
            setattr(clsobj, 'n_choices', 1)
        if not hasattr(clsobj, 'len_choice'):
            setattr(clsobj, 'len_choice', 2)
        if not hasattr(clsobj, 'animal'):
            setattr(clsobj, 'animal', False)
        if not hasattr(clsobj, 'desc'):
            setattr(clsobj, 'desc', 'missing desc')
            
        # Do some validation of the variables:
        if clsobj.len_choice < 1 or clsobj.len_choice > 4:
            raise TypeError(f'{name}: len_choice must be at least 1, at most 4')
        if clsobj.n_choices < 1 or clsobj.n_choices > 5:
            raise TypeError(f'{name}: n_choices must be at least 1, at most 5')
        if clsobj.animal and clsobj.len_choice != 2:
            raise TypeError(f'{name}: Conflicting settings: animal=True implies len_choice=2')
        if clsobj.match_index and clsobj.match_order:
            print(f'{name} (warning): match_index=True makes match_order redundant')

        # Declare the inner functions:
        
        def check_win(self, choices: list, draw: list) -> bool:
            if clsobj.n_choices != len(choices):
                raise TypeError(f'Expected {clsobj.n_choices} choices, got {len(choices)}')
            if not all([len(c) == clsobj.len_choice for c in choices]):
                raise TypeError(f'Choices must be all of length {clsobj.len_choice}')
            _draw =[d[-(clsobj.len_choice):] for d in draw]
            game_round = Round(choices=choices, draw=draw, animal=clsobj.animal)
            if clsobj.match_index:
                return game_round.check_exact()
            elif clsobj.match_order:
                return game_round.check_ordered()
            else:
                return game_round.check_present()

        def calc(self, bet: float, choices: list, draw: list) -> float:
            if self.check_win(choices, draw):
                return (bet * clsobj.prize) - bet
            else:
                return -1 * bet

        def random(self) -> list:
            '''Returns random picks for the corresponding game type'''
            high = 24 if clsobj.animal else 10**clsobj.len_choice-1
            r = np.random.randint(low=0, high=high, size=clsobj.n_choices)
            return [str(v).zfill(clsobj.len_choice) for v in r]

        # End inner functions. Inject them into the class:
        setattr(clsobj, 'random', random)
        setattr(clsobj, 'check_win', check_win)
        setattr(clsobj, 'calc', calc)
        setattr(clsobj, 'name', name)
        
        # Create one instance of the class and set it in a dict where
        # the class name is the key and the instance the value:
        GameMeta.__type_instances[name] = clsobj()
        
        # Return the class from the metaclass so the universe doesn't break:
        return clsobj
    
    @classmethod
    def get_type(cls, name):
        return cls.__type_instances.get(name)
    
    @classmethod
    def get_types(cls):
        return tuple(cls.__type_instances.values())

In [8]:
# Bellow is an example of how the metaclass is supposed to be used. The games
# listed bellow are the most common games in the Animal Game.

# Many more can be created by combining the settings.
# A few settings conflict with each other, but hopefully the metaclass will
# detect this and produce a warning or error as appropriate.


# Settings:

# desc: str
# A description or name of the type of game.

# animal: bool
# Is this bet on an animal? If not it is on a number.

# len_choice: int
# The number of digits in each choice. Bets on animals always have two digits
# because animals are represented by a zero padded two digit string.

# n_choices: int
# The number of picks for this kind of game.

# match_order: bool
# If true not only must the picks be present in the draw, they must also be appear 
# in the same order. This setting has no effect if n_choices=1.

# match_index: bool
# More strict than match_order, this one requires that the exact position of the picks
# be matched, not only the order. Overrides match_order.

# prize: float
# The payout to the player is calculated by multiplying the amount bet for this value in case
# of a win.

In [9]:
# DEFAULTS (set by GameMeta if not present):
#     desc = 'missing desc'
#     animal = False
#     match_order = False
#     match_index = False
#     n_choices = 1
#     len_choice = 2
#     prize = 2

class DQ(metaclass=GameMeta):
    desc = 'Dezena Qualquer'
    prize = 12.0

class DS(metaclass=GameMeta):
    desc = 'Dezena Seca'
    prize = 60.0
    match_index = True

class GQ(metaclass=GameMeta):
    desc = 'Grupo Qualquer'
    animal = True
    prize = 3.6

class GS(metaclass=GameMeta):
    desc = 'Grupo Seco'
    animal = True
    match_index = True
    prize = 18.0

class GD(metaclass=GameMeta):
    desc = 'Grupo Duque'
    animal = True
    prize = 18.5
    n_choices = 2

class GT(metaclass=GameMeta):
    desc = 'Grupo Terno'
    animal = True
    n_choices = 3
    prize = 130.0

class DD(metaclass=GameMeta):
    desc = 'Duque Dezenas'
    prize = 300.0
    n_choices = 2

class DT(metaclass=GameMeta):
    desc = 'Dezena Terno'
    n_choices = 3
    prize = 3000.0

class CQ(metaclass=GameMeta):
    desc = 'Centena Qualquer'
    len_choice = 3
    prize = 120.0

class CS(metaclass=GameMeta):
    desc = 'Centena Seca'
    len_choice = 3
    match_index = True
    prize = 600.0

class MQ(metaclass=GameMeta):
    desc = 'Milhar Qualquer'
    len_choice = 4
    prize = 800.0

class MS(metaclass=GameMeta):
    desc = 'Milhar Seco'
    len_choice = 4
    match_index = True
    prize = 4000.0


# Example of a made up kind of game:

#class TI(metaclass=GameMeta):
#    desc = 'Terno Improvável'
#    animal = True
#    n_choices = 3
#    match_order = True
#    prize = 1200.0

In [10]:
def bet_sample(mean, std, size):
    a = np.random.normal(loc=mean, scale=std, size=size)
    sample = np.around(a, decimals=1)
    return sample[sample > 0]

In [11]:
def random_draw():
    return [str(n).zfill(4) for n in np.random.randint(low=0, high=9999, size=5)]

In [12]:
house_pov = True

In [13]:
import pandas as pd

def make_simulation(bet_mean, bet_std, size, house_pov=True, draw=None):
    df = pd.DataFrame()
    game_types = GameMeta.get_types()
    df['bet'] = bet_sample(mean=bet_mean, std=bet_std, size=size)
    if draw is None:
        draw = random_draw()
    picks = []
    types = []
    codes = []
    net = []
    for bet in df['bet']:
        game = np.random.choice(game_types)
        types.append(game.desc)
        codes.append(game.name)
        pick = game.random()
        v = game.calc(bet=bet, draw=draw, choices=pick)
        if house_pov:
            v *= -1
        if game.animal:
            pick = [ANIMALS[int(i)] for i in pick]
        picks.append(pick)
        net.append(v)
    df['game_type'] = types
    df['code'] = codes
    df['picks'] = picks
    df['net'] = net
    df['draw'] = [draw for _ in range(len(df))]
    if not house_pov:
        df['bet'] = [-b for b in df['bet']]
    return df

In [14]:
import locale
locale.setlocale( locale.LC_ALL, '')

'LC_CTYPE=en_US.UTF-8;LC_NUMERIC=pt_BR.UTF-8;LC_TIME=en_US.UTF-8;LC_COLLATE=en_US.UTF-8;LC_MONETARY=pt_BR.UTF-8;LC_MESSAGES=en_US.UTF-8;LC_PAPER=pt_BR.UTF-8;LC_NAME=pt_BR.UTF-8;LC_ADDRESS=pt_BR.UTF-8;LC_TELEPHONE=pt_BR.UTF-8;LC_MEASUREMENT=pt_BR.UTF-8;LC_IDENTIFICATION=pt_BR.UTF-8'

In [15]:
def regroup_by_type(source, house_pov=True):
    df = pd.DataFrame()
    nets = source[['code', 'net']].groupby(['code']).sum()
    df['code'] = nets.index
    df['game_type'] = [GameMeta.get_type(s).desc for s in df['code']]
    df['n'] = source.groupby(['code']).count()['net'].values
    if house_pov:
        wins = source[source['net'] < 0].groupby(['code']).count()['picks']
    else:
        wins = source[source['net'] > 0].groupby(['code']).count()['picks']
    df['wins'] = [wins.get(k, 0) for k in df['code'].values]
    probs = [(a / b) * 100 for a, b in zip(df['wins'].values, df['n'].values)]
    df['p'] = [f'{prob:.3f}%' for prob in probs]
    df['net'] = [s for s in nets['net']]
    df.sort_values('net', inplace=True)
    avg = (df['wins'].sum() / df['n'].sum()) * 100
    df.loc[len(df)] = ['', 'TOTAL', f"{df['n'].sum():_}", f"{df['wins'].sum():_}", f'{avg:.3f}%', df['net'].sum()]
    df['net'] = [locale.currency(s, grouping=True) for s in df['net'].values]
    df['n'] = [f'{int(a):_}' for a in df['n']]
    df['wins'] = [f'{int(a):_}' for a in df['wins']]
    df.index = list(range(len(df)))
    return df

In [16]:
from contextlib import contextmanager
import time

@contextmanager
def timer():
    before = time.time()
    yield
    after = time.time()
    print(f'Block executed in {after - before:.2f} seconds')

In [17]:
lottery_draw = random_draw()
animal_names = [ANIMALS[int(animal_id_from_digits(n))] for n in lottery_draw]
print(animal_names)
lottery_draw

['Pavão', 'Cobra', 'Porco', 'Leão', 'Águia']


['2173', '3435', '7969', '2262', '0206']

In [18]:
with timer():
    sim = make_simulation(draw=lottery_draw, bet_mean=11, bet_std=5, size=200_200, house_pov=house_pov).sort_values('net')

Block executed in 9.99 seconds


In [19]:
sim_display = sim.head(12).copy()
sim_display['net'] = [locale.currency(n, grouping=True) for n in sim_display['net'].values]
sim_display['bet'] = [locale.currency(b, grouping=True) for b in sim_display['bet'].values]
sim_display
sim.tail(4)

,bet,game_type,code,picks,net,draw
133122,31.4,Milhar Qualquer,MQ,[9051],31.4,"[2173, 3435, 7969, 2262, 0206]"
11129,31.5,Centena Qualquer,CQ,[864],31.5,"[2173, 3435, 7969, 2262, 0206]"
87686,32.7,Grupo Duque,GD,"[Jacaré, Cabra]",32.7,"[2173, 3435, 7969, 2262, 0206]"
132109,33.4,Grupo Qualquer,GQ,[Cachorro],33.4,"[2173, 3435, 7969, 2262, 0206]"


In [20]:
with timer():
    by_type = regroup_by_type(source=sim, house_pov=house_pov)
by_type

Block executed in 0.09 seconds


,code,game_type,n,wins,p,net
0,GS,Grupo Seco,16_326,692,4.239%,"R$ 42.428,60"
1,GQ,Grupo Qualquer,16_484,3_407,20.669%,"R$ 46.349,92"
2,CS,Centena Seca,16_459,18,0.109%,"R$ 58.085,80"
3,GD,Grupo Duque,16_493,580,3.517%,"R$ 62.717,65"
4,CQ,Centena Qualquer,16_559,89,0.537%,"R$ 67.457,10"
5,DQ,Dezena Qualquer,16_667,871,5.226%,"R$ 72.248,40"
6,DD,Duque Dezenas,16_242,33,0.203%,"R$ 75.214,80"
7,DS,Dezena Seca,16_556,164,0.991%,"R$ 76.145,50"
8,GT,Grupo Terno,16_386,64,0.391%,"R$ 95.345,70"
9,DT,Dezena Terno,16_355,2,0.012%,"R$ 100.688,30"


In [21]:
hr = (sim['net'].sum() / sim['bet'].sum()) * 100
f'House return on bets: {hr:.2f}%'

'House return on bets: 42.28%'

In [22]:
last_type = GameMeta.get_types()[-1]
mask = sim['code'] == last_type.name
games = sim[mask].head(8).copy()
games['net'] = [locale.currency(n, grouping=True) for n in games['net'].values]
games['bet'] = [locale.currency(b, grouping=True) for b in games['bet'].values]
games

,bet,game_type,code,picks,net,draw
164348,"R$ 12,10",Milhar Seco,MS,[2173],"-R$ 48.387,90","[2173, 3435, 7969, 2262, 0206]"
100720,"R$ 2,70",Milhar Seco,MS,[2173],"-R$ 10.797,30","[2173, 3435, 7969, 2262, 0206]"
129849,"R$ 0,10",Milhar Seco,MS,[1224],"R$ 0,10","[2173, 3435, 7969, 2262, 0206]"
112851,"R$ 0,10",Milhar Seco,MS,[4585],"R$ 0,10","[2173, 3435, 7969, 2262, 0206]"
155635,"R$ 0,10",Milhar Seco,MS,[1215],"R$ 0,10","[2173, 3435, 7969, 2262, 0206]"
80366,"R$ 0,10",Milhar Seco,MS,[5809],"R$ 0,10","[2173, 3435, 7969, 2262, 0206]"
121229,"R$ 0,10",Milhar Seco,MS,[1233],"R$ 0,10","[2173, 3435, 7969, 2262, 0206]"
145607,"R$ 0,10",Milhar Seco,MS,[5467],"R$ 0,10","[2173, 3435, 7969, 2262, 0206]"


In [23]:
mask = sim['code'] == 'GT'
gt = sim[mask].copy()
gt['net'] = [locale.currency(s, grouping=True) for s in gt['net'].values]
gt['bet'] = [locale.currency(b, grouping=True) for b in gt['bet'].values]
gt.head(12)

,bet,game_type,code,picks,net,draw
17642,"R$ 20,50",Grupo Terno,GT,"[Porco, Cobra, Pavão]","-R$ 2.644,50","[2173, 3435, 7969, 2262, 0206]"
42101,"R$ 19,90",Grupo Terno,GT,"[Porco, Cobra, Pavão]","-R$ 2.567,10","[2173, 3435, 7969, 2262, 0206]"
26051,"R$ 19,80",Grupo Terno,GT,"[Águia, Porco, Pavão]","-R$ 2.554,20","[2173, 3435, 7969, 2262, 0206]"
78971,"R$ 19,60",Grupo Terno,GT,"[Cobra, Pavão, Águia]","-R$ 2.528,40","[2173, 3435, 7969, 2262, 0206]"
25802,"R$ 19,20",Grupo Terno,GT,"[Leão, Cobra, Pavão]","-R$ 2.476,80","[2173, 3435, 7969, 2262, 0206]"
184800,"R$ 18,70",Grupo Terno,GT,"[Cobra, Porco, Águia]","-R$ 2.412,30","[2173, 3435, 7969, 2262, 0206]"
51622,"R$ 18,50",Grupo Terno,GT,"[Porco, Cobra, Leão]","-R$ 2.386,50","[2173, 3435, 7969, 2262, 0206]"
117064,"R$ 18,30",Grupo Terno,GT,"[Porco, Cobra, Águia]","-R$ 2.360,70","[2173, 3435, 7969, 2262, 0206]"
29284,"R$ 18,10",Grupo Terno,GT,"[Leão, Porco, Cobra]","-R$ 2.334,90","[2173, 3435, 7969, 2262, 0206]"
152142,"R$ 17,80",Grupo Terno,GT,"[Leão, Pavão, Águia]","-R$ 2.296,20","[2173, 3435, 7969, 2262, 0206]"


In [24]:
animal_names

['Pavão', 'Cobra', 'Porco', 'Leão', 'Águia']